In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('data/skygeni_sales_data.csv')
df.sample(5)

,deal_id,created_date,closed_date,sales_rep_id,industry,region,product_type,lead_source,deal_stage,deal_amount,sales_cycle_days,outcome
3073,D03074,2024-01-05,2024-04-20,rep_13,Ecommerce,India,Core,Partner,Qualified,6802,106,Won
1780,D01781,2023-04-21,2023-07-25,rep_25,SaaS,APAC,Pro,Outbound,Qualified,3160,95,Lost
836,D00837,2024-01-21,2024-05-03,rep_1,SaaS,India,Pro,Inbound,Demo,43680,103,Won
1227,D01228,2024-03-20,2024-06-25,rep_19,EdTech,North America,Enterprise,Referral,Proposal,28573,97,Won
3710,D03711,2023-11-18,2024-03-17,rep_13,Ecommerce,India,Pro,Inbound,Closed,96119,120,Won


# 0. Problem Definition
- The CRO observes declining win rate despite stable pipeline volume.
- The goal is to identify which deal attributes (segment, region, ACV, cycle time, etc.) and sales behaviors are statistically associated with win probability, and to detect which factors contributed to the win rate decline in the last two quarters.

# 1. Feature Engineering

In [3]:
df["created_date"] = pd.to_datetime(df["created_date"])
df["closed_date"] = pd.to_datetime(df["closed_date"])

df["created_month"] = df["created_date"].dt.month
df["created_quarter"] = df["created_date"].dt.quarter
df["created_year"] = df["created_date"].dt.year
df['created_year_and_quarter'] = df["created_date"].dt.to_period("Q").astype(str)
df["created_weekday"] = df["created_date"].dt.weekday

df["closed_month"] = df["closed_date"].dt.month
df["closed_quarter"] = df["closed_date"].dt.quarter
df["closed_year"] = df["closed_date"].dt.year
df['closed_year_and_quarter'] = df["closed_date"].dt.to_period("Q").astype(str)
df["closed_weekday"] = df["closed_date"].dt.weekday

# Filter out the last quarter
latest_quarter = df['closed_year_and_quarter'].max()
df = df[df['closed_year_and_quarter'] != latest_quarter].copy()

# Time buckets (helps CRO thinking)
df["sales_cycle_bucket"] = pd.cut(
    df["sales_cycle_days"],
    bins=[0, 14, 30, 60, 90, np.inf],
    labels=["<2w", "2-4w", "1-2m", "2-3m", "3m+"]
)

# Binary outcome
df["is_won"] = (df["outcome"] == "Won").astype(int)

# ACV buckets (very important) --> Right skewed data, hence log transformed
df["acv_bucket"] = pd.qcut(
    df["deal_amount"], 
    # np.log1p(df["deal_amount"]), 
    q=4,
    labels=["Low", "Mid", "High", "Very High"]
)

# doesn't matter if I apply log or not in pd.qcut as log is a monotonic function (preserves order) 
# and qcut works by splitting thr ranked data into equal sized groups
# hence ranking order of values doesn't change post log applying
# NOTE: for strictly qcut into bucket applications, skip applying log as it is just additional compute, 
# look at examples/qcut-example.ipynb for sanity check

# 2. Detect Structural Changes in Win Rate Drivers

In [48]:
def segment_win_rate_drift_weighted(df, segment_col):
    """
    Computes win rate delta, weighted impact, and contribution percentage
    for a segment between baseline and decline periods.

    Contribution (%) = portion of weighted impact relative to total weighted impact

    Parameters:
    - df: DataFrame containing at least 'closed_year_and_quarter', 'is_won', 'deal_id', and segment_col
    - segment_col: column to analyze (e.g., 'sales_cycle_bucket', 'region', 'product_type')

    Returns:
    - pivot table with segment, baseline & decline win rates, delta, deal counts, 
      weighted impact, and contribution percentage
    """
    
    # Sort quarters chronologically
    quarters = sorted(df["closed_year_and_quarter"].unique())
    
    # Last 2 completed quarters = decline period
    decline_quarters = quarters[-2:]
    
    df["period_flag"] = np.where(
        df["closed_year_and_quarter"].isin(decline_quarters),
        "decline_period",
        "baseline_period"
    )
    
    # Aggregate deals and win rates by period and segment
    summary = (
        df.groupby(["period_flag", segment_col])
          .agg(
              deals=("deal_id", "count"),
              win_rate=("is_won", "mean")
          )
          .reset_index()
    )
    
    # Pivot to have baseline vs decline side by side
    pivot = summary.pivot_table(
        index=segment_col,
        columns="period_flag",
        values=["win_rate", "deals"]
    )
    
    # Flatten column names
    pivot.columns = ['_'.join(col).strip() for col in pivot.columns.values]
    pivot = pivot.reset_index()
    
    # Calculate win rate delta
    pivot["win_rate_delta"] = pivot["win_rate_decline_period"] - pivot["win_rate_baseline_period"]
    
    # Weighted impact = delta * deals in decline period
    pivot["weighted_impact"] = pivot["win_rate_delta"] * pivot["deals_decline_period"]
    
    # Only consider negative impacts for the drop
    total_negative_impact = -pivot[pivot["weighted_impact"] < 0]["weighted_impact"].sum()
    
    # Contribution pct for segments causing decline
    pivot["contribution_pct"] = pivot["weighted_impact"].apply(
        lambda x: (x / total_negative_impact * 100) if x < 0 else 0
    )

    # Sort by weighted impact (most negative first)
    return pivot.sort_values("weighted_impact")


In [49]:
region_drift = segment_win_rate_drift_weighted(df, "region")
industry_drift = segment_win_rate_drift_weighted(df, "industry")
acv_drift = segment_win_rate_drift_weighted(df, "acv_bucket")
cycle_drift = segment_win_rate_drift_weighted(df, "sales_cycle_bucket")

C:\Users\robot\AppData\Local\Temp\ipykernel_1968\186342618.py:31: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(["period_flag", segment_col])
C:\Users\robot\AppData\Local\Temp\ipykernel_1968\186342618.py:40: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot = summary.pivot_table(
C:\Users\robot\AppData\Local\Temp\ipykernel_1968\186342618.py:31: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(["period_flag", segment_col])
C:\Us

#### win_rate_delta = win_rate_decline_period - win_rate_baseline_period
#### weighted_impact = win_rate_delta * deals_decline_period
#### NOTE: Negative values in weighted_impact mean drop in win_rate and positive means improvement

In [54]:
region_drift

,region,deals_baseline_period,deals_decline_period,win_rate_baseline_period,win_rate_decline_period,win_rate_delta,weighted_impact,contribution_pct
0,APAC,836.0,397.0,0.458134,0.430730,-0.027403,-10.879187,-100.0
1,Europe,825.0,407.0,0.455758,0.457002,0.001245,0.506667,0.0
2,India,852.0,424.0,0.453052,0.457547,0.004496,1.906103,0.0
3,North America,831.0,399.0,0.435620,0.476190,0.040571,16.187726,0.0


In [55]:
industry_drift

,industry,deals_baseline_period,deals_decline_period,win_rate_baseline_period,win_rate_decline_period,win_rate_delta,weighted_impact,contribution_pct
1,EdTech,653.0,332.0,0.445636,0.424699,-0.020937,-6.950995,-100.0
2,FinTech,615.0,318.0,0.476423,0.481132,0.004709,1.497561,0.0
4,SaaS,665.0,328.0,0.448120,0.457317,0.009197,3.016541,0.0
0,Ecommerce,729.0,324.0,0.444444,0.456790,0.012346,4.000000,0.0
3,HealthTech,682.0,325.0,0.441349,0.458462,0.017113,5.561584,0.0


In [58]:
industry_drift

,industry,deals_baseline_period,deals_decline_period,win_rate_baseline_period,win_rate_decline_period,win_rate_delta,weighted_impact,contribution_pct
1,EdTech,653.0,332.0,0.445636,0.424699,-0.020937,-6.950995,-100.0
2,FinTech,615.0,318.0,0.476423,0.481132,0.004709,1.497561,0.0
4,SaaS,665.0,328.0,0.448120,0.457317,0.009197,3.016541,0.0
0,Ecommerce,729.0,324.0,0.444444,0.456790,0.012346,4.000000,0.0
3,HealthTech,682.0,325.0,0.441349,0.458462,0.017113,5.561584,0.0


In [56]:
acv_drift

,acv_bucket,deals_baseline_period,deals_decline_period,win_rate_baseline_period,win_rate_decline_period,win_rate_delta,weighted_impact,contribution_pct
0,Low,830.0,413.0,0.454217,0.452785,-0.001432,-0.591566,-100.0
2,High,822.0,420.0,0.458637,0.461905,0.003267,1.372263,0.0
1,Mid,856.0,387.0,0.427570,0.434109,0.006538,2.530374,0.0
3,Very High,836.0,407.0,0.462919,0.471744,0.008826,3.592105,0.0


In [57]:
cycle_drift

,sales_cycle_bucket,deals_baseline_period,deals_decline_period,win_rate_baseline_period,win_rate_decline_period,win_rate_delta,weighted_impact,contribution_pct
2,1-2m,929.0,382.0,0.442411,0.405759,-0.036652,-14.001076,-87.206811
1,2-4w,519.0,164.0,0.512524,0.500000,-0.012524,-2.053950,-12.793189
4,3m+,760.0,526.0,0.440789,0.441065,0.000275,0.144737,0.000000
0,<2w,274.0,72.0,0.441606,0.500000,0.058394,4.204380,0.000000
3,2-3m,862.0,483.0,0.433875,0.488613,0.054738,26.438515,0.000000


# 3. Bayesian Logistic Regressoin for Probabilistic outcome

In [59]:
model_df = df.copy()

In [60]:
# Select relevant features
features = [
    "region",
    "industry",
    "acv_bucket",
    "sales_cycle_bucket",
    "lead_source"
]

# One-hot encode categorical variables
model_df = pd.get_dummies(
    model_df[features + ["is_won"]],
    drop_first=True
)

X = model_df.drop(columns=["is_won"])
y = model_df["is_won"].values

feature_names = X.columns.tolist()
X = X.values

In [65]:
import pymc as pm
import arviz as az

with pm.Model() as bayesian_logistic_model:
    
    # Priors
    intercept = pm.Normal("intercept", mu=0, sigma=2)
    coefs = pm.Normal("coefs", mu=0, sigma=1, shape=X.shape[1])
    
    # Linear model
    logits = intercept + pm.math.dot(X, coefs)
    
    # Likelihood
    likelihood = pm.Bernoulli(
        "obs",
        logit_p=logits,
        observed=y
    )
    
    trace = pm.sample(
        1000,
        tune=1000,
        chains=4,
        target_accept=0.9,
        return_inferencedata=True
    )


 Progress                    Draws   Divergences   Step size   Grad evals   Sampling Speed   Elapsed   Remaining  
 ───────────────────────────────────────────────────────────────────────────────────────────────────────────────── 
  ━━━━━━━━━━━━━━━━━━━━━━━━━   2000    0             0.194       15           24.29 draws/s    0:01:22   0:00:00    
  ━━━━━━━━━━━━━━━━━━━━━━━━━   2000    0             0.241       15           25.50 draws/s    0:01:18   0:00:00    
  ━━━━━━━━━━━━━━━━━━━━━━━━━   2000    0             0.176       15           25.72 draws/s    0:01:17   0:00:00    
  ━━━━━━━━━━━━━━━━━━━━━━━━━   2000    0             0.215       31           12.98 draws/s    0:02:34   0:00:00

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 172 seconds.


In [66]:
az.summary(trace, var_names=["intercept", "coefs"])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
intercept,-0.184,0.140,-0.430,0.096,0.004,0.002,1496.0,2422.0,1.0
coefs[0],0.031,0.080,-0.113,0.185,0.002,0.001,2454.0,2920.0,1.0
coefs[1],0.025,0.079,-0.121,0.173,0.002,0.001,2686.0,2924.0,1.0
coefs[2],0.001,0.081,-0.156,0.147,0.001,0.001,2912.0,3204.0,1.0
coefs[3],-0.045,0.088,-0.212,0.119,0.002,0.001,3008.0,3380.0,1.0
coefs[4],0.115,0.089,-0.043,0.288,0.002,0.001,3036.0,3216.0,1.0
coefs[5],-0.006,0.090,-0.185,0.155,0.002,0.001,2881.0,2686.0,1.0
coefs[6],0.011,0.088,-0.157,0.171,0.002,0.001,2922.0,2742.0,1.0
coefs[7],-0.094,0.083,-0.249,0.061,0.002,0.001,2823.0,2690.0,1.0
coefs[8],0.026,0.081,-0.122,0.183,0.001,0.001,2921.0,2728.0,1.0


In [67]:
coef_summary = az.summary(trace, var_names=["coefs"])

coef_summary["feature"] = feature_names
coef_summary = coef_summary.sort_values("mean")

coef_summary


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat,feature
coefs[7],-0.094,0.083,-0.249,0.061,0.002,0.001,2823.0,2690.0,1.0,acv_bucket_Mid
coefs[15],-0.091,0.080,-0.235,0.065,0.002,0.001,2477.0,2826.0,1.0,lead_source_Partner
coefs[11],-0.078,0.114,-0.300,0.136,0.003,0.002,1666.0,2021.0,1.0,sales_cycle_bucket_1-2m
coefs[13],-0.048,0.113,-0.265,0.163,0.003,0.002,1647.0,1987.0,1.0,sales_cycle_bucket_3m+
coefs[3],-0.045,0.088,-0.212,0.119,0.002,0.001,3008.0,3380.0,1.0,industry_EdTech
coefs[14],-0.029,0.081,-0.195,0.110,0.002,0.001,2816.0,2912.0,1.0,lead_source_Outbound
coefs[16],-0.022,0.080,-0.174,0.124,0.002,0.001,2803.0,3055.0,1.0,lead_source_Referral
coefs[5],-0.006,0.090,-0.185,0.155,0.002,0.001,2881.0,2686.0,1.0,industry_HealthTech
coefs[2],0.001,0.081,-0.156,0.147,0.001,0.001,2912.0,3204.0,1.0,region_North America
coefs[12],0.010,0.114,-0.201,0.222,0.003,0.002,1658.0,2273.0,1.0,sales_cycle_bucket_2-3m


In [68]:
coef_summary["odds_ratio"] = np.exp(coef_summary["mean"])

coef_summary["probability_lift"] = (
    coef_summary["odds_ratio"] - 1
)

coef_summary.sort_values("probability_lift")


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat,feature,odds_ratio,probability_lift
coefs[7],-0.094,0.083,-0.249,0.061,0.002,0.001,2823.0,2690.0,1.0,acv_bucket_Mid,0.910283,-0.089717
coefs[15],-0.091,0.080,-0.235,0.065,0.002,0.001,2477.0,2826.0,1.0,lead_source_Partner,0.913018,-0.086982
coefs[11],-0.078,0.114,-0.300,0.136,0.003,0.002,1666.0,2021.0,1.0,sales_cycle_bucket_1-2m,0.924964,-0.075036
coefs[13],-0.048,0.113,-0.265,0.163,0.003,0.002,1647.0,1987.0,1.0,sales_cycle_bucket_3m+,0.953134,-0.046866
coefs[3],-0.045,0.088,-0.212,0.119,0.002,0.001,3008.0,3380.0,1.0,industry_EdTech,0.955997,-0.044003
coefs[14],-0.029,0.081,-0.195,0.110,0.002,0.001,2816.0,2912.0,1.0,lead_source_Outbound,0.971416,-0.028584
coefs[16],-0.022,0.080,-0.174,0.124,0.002,0.001,2803.0,3055.0,1.0,lead_source_Referral,0.978240,-0.021760
coefs[5],-0.006,0.090,-0.185,0.155,0.002,0.001,2881.0,2686.0,1.0,industry_HealthTech,0.994018,-0.005982
coefs[2],0.001,0.081,-0.156,0.147,0.001,0.001,2912.0,3204.0,1.0,region_North America,1.001001,0.001001
coefs[12],0.010,0.114,-0.201,0.222,0.003,0.002,1658.0,2273.0,1.0,sales_cycle_bucket_2-3m,1.010050,0.010050


### Focusing on last 2 Quarters

In [69]:
model_df["decline_period"] = (
    df["period_flag"] == "decline_period"
).astype(int)

# Interaction example:
for col in feature_names:
    model_df[f"{col}_decline_interaction"] = (
        model_df[col] * model_df["decline_period"]
    )


In [26]:
summary = (
    df.groupby(["period_flag", 'region'])
      .agg(
          deals=("deal_id", "count"),
          win_rate=("is_won", "mean")
      )
      .reset_index()
)

summary


,period_flag,region,deals,win_rate
0,baseline_period,APAC,836,0.458134
1,baseline_period,Europe,825,0.455758
2,baseline_period,India,852,0.453052
3,baseline_period,North America,831,0.435620
4,decline_period,APAC,397,0.430730
5,decline_period,Europe,407,0.457002
6,decline_period,India,424,0.457547
7,decline_period,North America,399,0.476190


In [27]:
pivot = summary.pivot_table(
    index='region',
    columns="period_flag",
    values=["win_rate", "deals"]
).reset_index()

# Flatten column names
pivot.columns = ['_'.join(col).strip() for col in pivot.columns.values]
pivot = pivot.reset_index()

pivot["win_rate_delta"] = (
    pivot["decline_period"] - pivot["baseline_period"]
)
pivot

KeyError: 'decline_period'